# truss-me.ipynb
This notebook demonstrates a simple finite elements program written entirely in Python

In [ ]:
%matplotlib widget
import numpy as np

from bar import Bar
from truss import Truss
import plotting as plots

## Defining Element Properties

In [ ]:
# Define node positions (m)
rNodes = 1e-2 * np.array([
    [-95.25, 0., 508],
    [95.25, 0., 508],
    [-95.25, 95.25, 254],
    [95.25, 95.25, 254],
    [95.25, -95.25, 254],
    [-95.25, -95.25, 254],
    [-254., 254, 0],
    [254., 254, 0],
    [254., -254, 0],
    [-254., -254, 0]
])
nNodes = len(rNodes)

# Define element nodes (2 per Bar element)
eNodes = np.array([
    [0,1], [0,3], [1,2], [0,4], [1,5],
    [1,3], [1,4], [0,2], [0,5], [2,5],
    [3,4], [2,3], [4,5], [2,9], [5,6],
    [3,8], [4,7], [3,6], [2,7], [4,9],
    [5,8], [5,9], [2,6], [4,8], [3,7],
])
nEl = len(eNodes)

# Define element areas (m^2), Young's moduli (Pa), & densities (kg/m^3)
elAs = 1e-4 * np.array([
    .213, 13, 13, 13, 13,
    18.213, 18.213, 18.213, 18.213, 0.065,
    0.065, 0.09, 0.09, 6.323, 6.323,
    6.323, 6.323, 11.355, 11.355, 11.355,
    11.355, 15.742, 15.742, 15.742, 15.742,
])
elYs = np.full(nEl, 6.89e10)
elDens = np.full(nEl, 2.7e3)

## Define Forcing Conditions
Our next step is to define the forcing conditions we wish to apply to our truss structure (including any fixed nodes which are not free to react to applied forces).

In [ ]:
# Define nodal degrees of freedom (1:free; 0:fixed)
fixNodes = np.array([[3*i, 3*i+1, 3*i+2] for i in [6, 7, 8, 9]]).flatten()
DOF = np.array([1 for i in range(3*nNodes)])
DOF[fixNodes] = 0

# Define applied nodal forces (N) as a flat vector
FNodes = [0, 1, 2, 5]
FVals = np.array([
    [4448.222, 44482.216,-22241.108],
    [0., 44482.216, -22241.108],
    [2224.111, 0., 0.],
    [2224.111, 0., 0.]
])
nFs = np.zeros((nNodes,3))
nFs[FNodes] = FVals
nFs = nFs.flatten()

## Create a Truss & Apply Forces
Finally we can create a truss structure using node and element information, and solve the elastodynamic equations to retrieve displacements and strains in the structure.

In [ ]:
truss = Truss(elYs, elDens, elAs, rNodes, eNodes)
truss.applyForces(DOF,nFs)

## Plot Results

In [ ]:
plots.plot_struct(truss)
plots.plot_disps(truss, magnify=100)
plots.plot_stress(truss)